In [ ]:
!earthengine authenticate

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ee
import geemap
ee.Initialize()

In [ ]:
#----------------- MAP VISUALIZER ---------------------
scan_loc = [-77.93,40.72] # station coordinates (long,lat)
site_name = "Rock Springs, PA"
start_date = '2024-01-01'
end_date = '2024-12-31'
# SCAN site location
point = ee.Geometry.Point(scan_loc)
# Load SMAP
smap_collection = ee.ImageCollection("NASA/SMAP/SPL3SMP_E/006") \
  .filterDate(start_date, end_date) \
  .filterBounds(point) \

# Load one SMAP image from the collection
sample_image = smap_collection.first()

# Get the projection from the image
projection = sample_image.select('soil_moisture_am').projection()

# SMAP GSD
pixel_bounds = point.buffer(4500).bounds()

# PRISM grid
prism_bounds = point.buffer(4638.3/2).bounds()

# Visualize with geemap
import geemap
Map = geemap.Map()
Map.center_object(point, 8)
Map.addLayer(pixel_bounds, {'color': 'red'}, 'SMAP Approximation')
Map.addLayer(prism_bounds, {'color': 'blue'}, 'PRISM Approximation')
Map.addLayer(point, {'color': 'green'}, 'SCAN Site')
Map


# Data Extraction
 - Pull data from SCAN sites stored in local CSV files
 - Pull SMAP data from Google Earth Engine
 - Pull PRISM meterological data from Google Earth Engine

In [ ]:

#--------------Map function for extracting soil moisture data---------------
def extract_soil_moisture(image,point):

    # Extract dates
    date = ee.Date(image.date()).format('YYYY-MM-dd')

    # Extract soil moisture value (PM)
    sm_value = image.select('soil_moisture_pm').reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=10000
    ).get('soil_moisture_pm')

    # Extract Weighted average of horizontally polarized brightness temperatures (PM)
    bth_value = image.select('tb_h_corrected_pm').reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=10000
    ).get('tb_h_corrected_pm')

    # Extract Weighted average of vertically polarized brightness temperatures (PM)
    btv_value = image.select('tb_v_corrected_pm').reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=10000
    ).get('tb_v_corrected_pm')

    # Extract quality flags (PM)
    qual_flag_v = image.select('tb_qual_flag_v_pm').reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=10000
    ).get('tb_qual_flag_v_pm')

    qual_flag_h = image.select('tb_qual_flag_h_pm').reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=10000
    ).get('tb_qual_flag_h_pm')

    return image.set('date', date).set('sm_value', sm_value).set('bth_value', bth_value).set('btv_value', btv_value).set('qual_flag_v', qual_flag_v).set('qual_flag_h', qual_flag_h)


# ----------------------- FUNCTION TO IMPORT SMAP DATA -------------------------
def smap_extractor(scan_loc,start_date,end_date):
  # SCAN site location
  point = ee.Geometry.Point(scan_loc)

  # Load SMAP
  smap_collection = ee.ImageCollection("NASA/SMAP/SPL3SMP_E/006") \
    .filterDate(start_date, end_date) \
    .filterBounds(point) \

  # Print number of images in the collection
  print(f"Number of images in the collection before filtering: {smap_collection.size().getInfo()}")

  # Map over the image collection and filter out None values
  smap_features = smap_collection.map(lambda image: extract_soil_moisture(image, point)).filter(ee.Filter.notNull(['sm_value', 'qual_flag_v','qual_flag_h'])) # Pass 'point' to extract_soil_moisture

  # Filter to only the samples that have good quality for brightness temperature
  smap_features = smap_features.filter(
      ee.Filter.And(
          ee.Filter.eq('qual_flag_v', 0),
          ee.Filter.eq('qual_flag_h', 0)
      )
  )

  # Number of images in the collection after filtering
  count = smap_features.size()
  print(f"Number of images in the collection after filtering: {count.getInfo()}")

  # Extract the time series data by aggregate data
  sm_values = smap_features.aggregate_array('sm_value').getInfo()

  # Extract corresponding dates
  dates = smap_features.aggregate_array('date').getInfo()

  # Extract brightness temperatures
  bth_values = smap_features.aggregate_array('bth_value').getInfo()
  btv_values = smap_features.aggregate_array('btv_value').getInfo()

  # Print lengths to check if they match
  print(f"Length of SM values: {len(sm_values)}")
  print(f"Length of Dates: {len(dates)}")
  print(f"Length of Brightness Temperature (Horizontal): {len(bth_values)}")
  print(f"Length of Brightness Temperature (Vertical): {len(btv_values)}")

  return dates, sm_values, bth_values, btv_values

#--------------Map function for extracting prism weather data---------------
def extract_weather_data(image,point):

    # Extract dates
    date = ee.Date(image.date()).format('YYYY-MM-dd')

    # Extract Daily total precipitation [mm]
    precip_value = image.select('ppt').reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=5000
    ).get('ppt')

    # Extract daily mean temperature [C]
    temp_value = image.select('tmean').reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=5000
    ).get('tmean')

    # Extract Daily mean dew point temperature [C]
    dew_value = image.select('tdmean').reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=5000
    ).get('tdmean')

    # Extract Daily minimum vapor pressure deficit [hPa]
    vpdmin_value = image.select('vpdmin').reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=5000
    ).get('vpdmin')

    # Extract Daily maximum vapor pressure deficit [hPa]
    vpdmax_value = image.select('vpdmax').reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point,
        scale=5000
    ).get('vpdmax')

    return image.set('date', date).set('precip_value', precip_value).set('temp_value', temp_value).set('dew_value', dew_value).set('vpdmin_value', vpdmin_value).set('vpdmax_value', vpdmax_value)

# ----------------------- FUNCTION TO IMPORT PRISM DATA -------------------------
def prism_extractor(scan_loc,start_date,end_date):

    # SCAN site location
    point = ee.Geometry.Point(scan_loc)

    prism_collection = ee.ImageCollection("OREGONSTATE/PRISM/AN81d")\
        .filterDate(start_date, end_date) \
        .filterBounds(point)

    # Print number of images in the collection
    print(f"Number of images in the collection: {prism_collection.size().getInfo()}")

    # Pull weather data from each image
    prism_features = prism_collection.map(lambda image: extract_weather_data(image, point))

    # Extract dates
    dates = prism_features.aggregate_array('date').getInfo()

    # Extract the time series data by aggregate data
    precip_values = prism_features.aggregate_array('precip_value').getInfo()

    # Extract mean daily temperature values
    temp_values = prism_features.aggregate_array('temp_value').getInfo()

    # Extract mean daily dewpoint values
    dew_values = prism_features.aggregate_array('dew_value').getInfo()

    # Extract vapor pressure deficit values
    vpdmin_values = prism_features.aggregate_array('vpdmin_value').getInfo()
    vpdmax_values = prism_features.aggregate_array('vpdmax_value').getInfo()

    return dates, precip_values, temp_values, dew_values, vpdmin_values, vpdmax_values


# ---------------- FUNCTION TO PLOT TIMESERIES AT LOCATION ---------------------

def plot_all_time_series(merged_df, site_name):
    # First figure: Soil Moisture
    fig1, ax1 = plt.subplots(figsize=(12, 5))
    plt.plot(merged_df['date'], merged_df['sm_smap'], label='SMAP')
    plt.plot(merged_df['date'], merged_df['soil_moisture'], label='In-situ')
    plt.xlabel('Date')
    plt.ylabel('Soil Moisture [%]')
    plt.title(f'Soil Moisture Time Series (SMAP vs In-situ): {site_name}')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Second figure: 2x2 grid for the rest
    fig2, axs = plt.subplots(2, 2, figsize=(12, 5))

    # Plot 1: Brightness Temperature
    axs[0, 0].plot(merged_df['date'], merged_df['bth'], label='Horizontal')
    axs[0, 0].plot(merged_df['date'], merged_df['btv'], label='Vertical')
    axs[0, 0].set_ylabel('Brightness Temp [K]')
    axs[0, 0].set_title(f'Brightness Temperature (SMAP)')
    axs[0, 0].legend()
    axs[0, 0].grid(True)

    # Plot 2: Precipitation
    axs[0, 1].plot(merged_df['date'], merged_df['precip'], color='blue')
    axs[0, 1].set_ylabel('Precipitation [mm]')
    axs[0, 1].set_title(f'Precipitation (PRISM)')
    axs[0, 1].grid(True)

    # Plot 3: Temperature and Dew Point
    axs[1, 0].plot(merged_df['date'], merged_df['temp'], label='Temperature')
    axs[1, 0].plot(merged_df['date'], merged_df['dew'], label='Dew Point')
    axs[1, 0].set_ylabel('Temperature [°C]')
    axs[1, 0].set_title(f'Temperature (PRISM)')
    axs[1, 0].legend()
    axs[1, 0].grid(True)

    # Plot 4: Vapor Pressure Deficit
    axs[1, 1].plot(merged_df['date'], merged_df['vpdmin'], label='VPD Minimum')
    axs[1, 1].plot(merged_df['date'], merged_df['vpdmax'], label='VPD Maximum')
    axs[1, 1].set_ylabel('VPD [hPa]')
    axs[1, 1].set_title(f'Vapor Pressure Deficit (PRISM)')
    axs[1, 1].legend()
    axs[1, 1].grid(True)

    # X-axis labels for bottom plots
    for ax in axs[1, :]:
        ax.set_xlabel('Date')

    plt.suptitle(f'Time Series Data for {site_name}', fontsize=16)
    plt.tight_layout()
    plt.show()

# ------------------ FUNCTION TO BUILD ALL MOISTURE DATA ---------------------
def build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path):

    # Import SCAN Data (ground truth)
    sm_scan = pd.read_csv(scan_path)

    # Convert 'date' column to datetime objects
    sm_scan['date'] = pd.to_datetime(sm_scan['date'])

    # Import SMAP data for [Date, Soil Moisture, Brightness Temp. Horizontal, Brightness Temp. Vertical] at SCAN coordinates
    smap_dates, sm_smap, bth_values, btv_values = smap_extractor(scan_loc,start_date,end_date)

    # Import PRISM data for [Date, Precipitation, Temperature, Dew Point, Vapor Pressure Min, Vapor Pressure Max] at SCAN coordinates
    prism_dates, precip_values, temp_values, dew_values, vpdmin_values, vpdmax_values = prism_extractor(scan_loc,start_date,end_date)

    # ------------ PLOT TIME SERIES -----------------

    # Convert SMAP data to dataframe
    smap_df = pd.DataFrame({
        'date': smap_dates,
        'sm_smap': [val * 100 for val in sm_smap],  # scale values by 100
        'bth': bth_values,
        'btv': btv_values
    })
    smap_df['date'] = pd.to_datetime(smap_df['date'])

    # Convert PRISM data to dataframe
    prism_df = pd.DataFrame({
        'date': prism_dates,
        'precip': precip_values,
        'temp': temp_values,
        'dew': dew_values,
        'vpdmin': vpdmin_values,
        'vpdmax': vpdmax_values
    })
    prism_df['date'] = pd.to_datetime(prism_df['date'])

    # Merge SMAP and PRISM first
    merged_df = pd.merge(sm_scan, smap_df, on='date', how='inner')

    # Then merge the result with SMAP-SCAN
    merged_df = pd.merge(merged_df, prism_df, on='date', how='inner')

    # Check for empty values in SCAN data and remove
    if merged_df.isnull().values.any() == True:
       nullcount = merged_df.isnull().sum().sum()
       print(f"Found {nullcount} empty values in SCAN data. Removing corresponding rows...")
       merged_df = merged_df.dropna(subset=['soil_moisture'])


    # Plot time series data
    plot_all_time_series(merged_df, site_name)

    # Save to csv
    merged_df.to_csv('new_data.csv', index=False)

    print(merged_df.head())
    return merged_df

In [ ]:
# --------------------- IMPORT DATA FROM LOVELL SUMMIT, NV ---------------------

# SCAN site location
scan_loc = [-115.61,36.17] # station coordinates (long,lat)
site_name = "Lovell Summit, NV"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/lovell_summit_20-25.csv'

lovell_summit_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# --------------------- IMPORT DATA FROM ROCK SPRINGS, PA ---------------------

# SCAN site location
scan_loc = [-77.93,40.72] # station coordinates (long,lat)
site_name = "Rock Springs, PA"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/rock_springs_20-25.csv'

rock_springs_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# --------------------- IMPORT DATA FROM Wakulla #1, FL ---------------------

# SCAN site location
scan_loc = [-84.42,30.31] # station coordinates (long,lat)
site_name = "Wakulla #1, FL"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/wakulla_20-25.csv'

wakulla_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# --------------------- IMPORT DATA FROM Dee River Ranch, AL ---------------------
# SCAN site location
scan_loc = [-84.42,30.31] # station coordinates (long,lat)
site_name = "Dee River Ranch, AL"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/dee_river_ranch_20-25.csv'

dee_river_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# --------------------- IMPORT DATA FROM Ames, IA ---------------------
# SCAN site location
scan_loc = [-93.73,	42.02] # station coordinates (long,lat)
site_name = "Ames, IA"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/ames_20-25.csv'

ames_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# --------------------- IMPORT DATA FROM Lindsay, MT ---------------------
# SCAN site location
scan_loc = [-105.19,	47.21] # station coordinates (long,lat)
site_name = "Lindsay, MT"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/lindsay_20-25.csv'

lindsay_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# ------------------- IMPORT DATA FROM Mount Mansfield, VT ---------------------
# SCAN site location
scan_loc = [-72.83, 44.54] # station coordinates (long,lat)
site_name = "Mount Mansfield, VT"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/mount_mansfield_20-25.csv'

mount_mansfield_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# ------------------- IMPORT DATA FROM Powell Gardens, MO ---------------------
# SCAN site location
scan_loc = [-94.03, 38.87] # station coordinates (long,lat)
site_name = "Powell Gardens, MO"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/powell_gardens_20-25.csv'

powell_gardens_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# ------------------- IMPORT DATA FROM Tule Valley, UT ---------------------
# SCAN site location
scan_loc = [-113.46, 39.24] # station coordinates (long,lat)
site_name = "Tule Valley, UT"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/tule_valley_20-25.csv'

tule_valley_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# ------------------- IMPORT DATA FROM Schell-Osage, MO ---------------------
# SCAN site location
scan_loc = [-94.04, 37.99] # station coordinates (long,lat)
site_name = "Schell-Osage, MO"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/schell_osage_20-25.csv'

schell_osage_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# ------------------- IMPORT DATA FROM Ku-Nesa, KS ---------------------
# SCAN site location
scan_loc = [-95.19,	39.05] # station coordinates (long,lat)
site_name = "Ku-Nesa, KS"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/ku_nesa_20-25.csv'

ku_nesa_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# ------------------- IMPORT DATA FROM Conrad Ag Rc, MT ---------------------
# SCAN site location
scan_loc = [-111.92, 48.3] # station coordinates (long,lat)
site_name = "Conrad Ag Rc, MT"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/conrad_ag_20-25.csv'

conrad_ag_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# ------------------- IMPORT DATA FROM UW Platteville, WI ---------------------
# SCAN site location
scan_loc = [-90.39, 42.71] # station coordinates (long,lat)
site_name = "Platteville, WI"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/uw_platteville_20-25.csv'

uw_platteville_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# ------------------- IMPORT DATA FROM Starkville, MS ---------------------
# SCAN site location
scan_loc = [-88.78, 33.47] # station coordinates (long,lat)
site_name = "Starkville, MS"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/starkville_20-25.csv'

starkville_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# ------------------- IMPORT DATA FROM CMRB LTAR-MO, MO ---------------------
# SCAN site location
scan_loc = [-92.12, 39.23] # station coordinates (long,lat)
site_name = "CMRB LTAR-MO, MO"
start_date = '2020-01-01'
end_date = '2024-12-31'
scan_path = '/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/SCAN Data/crmb_ltar_20-25.csv'

crmb_ltar_df = build_moisture_data(scan_loc,site_name,start_date,end_date,scan_path)

In [ ]:
# ------------------- APPEND NEW DATA TO EXISTING DATASET ----------------------
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/Extracted Data/soil_moisture_data.csv', parse_dates=['date'])
new_data = pd.read_csv('/content/new_data.csv', parse_dates=['date'])

result = pd.concat([data, new_data], ignore_index=True)

# Save to csv
result.to_csv('/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/Extracted Data/soil_moisture_data.csv', index=False)

print(result.shape)

# Exploratory Data Analysis (EDA)

In [ ]:
# EDA FUNCTIONS
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import seaborn as sns
from scipy.stats import skew, kurtosis

# ----------- Calculate Band Statistics Function----------
def calculate_band_statistics(data):

    if len(data.shape) > 2:
        data = data.reshape(-1, data.shape[2])

    data_mean = np.mean(data,axis=0)
    data_std = np.std(data,axis=0)
    data_min = np.min(data,axis=0)
    data_max = np.max(data,axis=0)
    data_q1 = np.percentile(data,25,axis=0)
    data_median = np.median(data,axis=0)
    data_q3 = np.percentile(data,75,axis=0)
    data_skew = skew(data,axis=0)
    data_kurt = kurtosis(data,axis=0)

    stats = np.vstack([data_mean,data_std,data_min,data_max,data_q1,data_median,data_q3,data_skew,data_kurt]).T

    return stats

  # ----------------Correlation Function--------------------
def correlation_matrix(data,wl,stats):

    # reshape data where rows and pixels and cols are bands
    if len(data.shape) > 2:
        reshaped_data = data.reshape(-1, data.shape[2])
    else:
        reshaped_data = data

    # Calculate the covariance matrix between each band
    cor = np.zeros((reshaped_data.shape[1],reshaped_data.shape[1]))

    for i in range(reshaped_data.shape[1]):
        for j in range(reshaped_data.shape[1]):
            cor[i,j] = ((np.sum((reshaped_data[:,i] - stats[i,0]) * (reshaped_data[:,j] - stats[j,0]))) / reshaped_data.shape[0]) / (stats[i,1]*stats[j,1])

    # Round to 3 decimal places
    cor_format = np.round(cor, 3)

    # Display the array as a heatmap
    fig, ax = plt.subplots(figsize=(6,5))
    cax = ax.imshow(cor_format, cmap='coolwarm',vmin=-1, vmax=1)

    # Add a colorbar
    plt.colorbar(cax)

    # Set evenly spaced tick locations
    max_ticks = 15  # Maximum number of ticks to display
    num_bands = cor_format.shape[0]  # Number of spectral bands

    # Dont set more ticks than available bands
    num_ticks = min(max_ticks, num_bands)

    # Generate tick positions dynamically
    tick_positions = np.linspace(0, num_bands - 1, num=num_ticks, dtype=int)
    ax.set_xticks(tick_positions)
    ax.set_yticks(tick_positions)

    # Set labels corresponding to selected tick positions
    ax.set_xticklabels([f"{wl[i]}" for i in tick_positions], rotation=45)
    ax.set_yticklabels([f"{wl[i]}" for i in tick_positions], rotation=45)

    plt.title("Correlation Matrix")
    plt.xlabel("Features")
    plt.ylabel("Features")
    plt.show()

    return cor

# -------Function to compute feature/target correlation--------
def feature_target_correlation(data,target):
    # reshape data where rows and pixels and cols are bands
    if len(data.shape) > 2:
        reshaped_data = data.reshape(-1, data.shape[2])
    target = target.reshape(-1, 1)
    r = np.sum((data - data.mean(axis=0)) * (target.reshape(-1, 1) - target.mean()), axis=0) / \
        np.sqrt(np.sum((data - data.mean(axis=0)) ** 2, axis=0) * np.sum((target - target.mean()) ** 2))
    return r

# ----------- Plot Band Statistics Function----------
def stats_plot(stats,band_names):

    # Format the matrix to show only 3 significant digits
    stats_format = np.round(stats, 3)  # Round to 3 decimal places
    stats_format = stats_format.astype(str)  # Convert to string for table
    col_labels = ["Mean", "Std", "Min", "Max", "Q1", 'Median', 'Q3', 'Skew', 'Kurt']

    # Show statistics as a table
    fig, ax = plt.subplots()
    ax.axis("tight")
    ax.axis("off")

    # Add the table
    table = ax.table(cellText=stats_format, loc="center", cellLoc="center",rowLabels=band_names, colLabels=col_labels)
    table.auto_set_font_size(False)
    table.set_fontsize(7)

    plt.title("Band Statistics", loc='center', fontsize=16, pad=0.1)
    plt.show()

# ----------- Feature Target Correlation Density Plot Function----------
def feature_target_density(data,target):
  import matplotlib.pyplot as plt
  import numpy as np
  from scipy.stats import gaussian_kde

  # plot relationship of each feature with target color by density
  fig, ax = plt.subplots(2, 4, figsize=(12, 6))
  axes = ax.flatten()

  for i in range(len(feature_names)):
      x = features[:, i]
      y = target

      # Calculate point density
      xy = np.vstack([x, y])
      z = gaussian_kde(xy)(xy)

      sc = axes[i].scatter(x, y, c=z, s=15, edgecolors='none', cmap='viridis')
      axes[i].set_title(f'{feature_names[i]}')
      axes[i].set_xlabel('Feature Value')
      axes[i].set_ylabel('Soil Moisture')

  plt.suptitle('Relationship of Soil Moisture Per Feature')
  plt.tight_layout()
  plt.show()



In [ ]:
# --------------------------- PERFORM EDA --------------------------------

# Read in extracted soil moisture data
data = pd.read_csv('/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/Extracted Data/soil_moisture_data.csv', parse_dates=['date'])

# Scale soil moisture values from percentage to volume ratio
data['soil_moisture'] = data['soil_moisture'] / 100
data['sm_smap'] = data['sm_smap'] / 100

# Separate into target and features
date = data['date']
target = data['soil_moisture'].to_numpy()
reference = data['sm_smap'].to_numpy()
features = data.drop(columns=['soil_moisture', 'date','sm_smap'])
feature_names = list(features.columns)
features = features.to_numpy()
print(len(feature_names))

print("Target Shape", target.shape)
print("Features Shape", features.shape)

# Histogram of soil moisture
plt.hist(target, bins=50,edgecolor='black')
plt.xlabel('Soil Moisture')
plt.ylabel('Frequency')
plt.title('Histogram of Soil Moisture')
plt.show()

# Calculate band statistics
stats = calculate_band_statistics(features)

# Plot the statistics of each feature
stats_plot(stats, feature_names)

# plot correlation with target
r = feature_target_correlation(features,target)
plt.bar(feature_names, r,edgecolor='black')
plt.title("Feature-Target Correlation")
plt.xlabel("Feature Name")
plt.ylabel("Correlation")
plt.show()

# plot relationship of each feature with target
feature_target_density(features,target)

# Show correlation matrix
print(list(range(features.shape[1])))
cor = correlation_matrix(features,feature_names,stats)


In [ ]:
#-------------------------- DATA PRE-PROCESSING ---------------------------------

# Make temp array so we can compare with SMAP predictions
temp_array = np.hstack((features, reference.reshape(-1,1)))

# Divide into training and testing splits
X_train, X_test, y_train, y_test = train_test_split(temp_array, target, test_size=0.2, random_state=42)

# Pull off the SMAP predictions to use later
ref_train = X_train[:, -1]
ref_test = X_test[:, -1]
X_train = X_train[:, :-1]
X_test = X_test[:, :-1]

# Standardize data according to training
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Training Data Shape", X_train.shape)
print("Testing Data Shape", X_test.shape)

# Histogram of splits
fig, ax = plt.subplots(1,2,figsize=(10,4))
ax[0].hist(y_train, bins=50,edgecolor='black')
ax[0].set_xlabel('Soil Moisture %')
ax[0].set_ylabel('Frequency')
ax[0].set_title('Histogram of Soil Moisture (Training)')
ax[1].hist(y_test, bins=50,edgecolor='black')
ax[1].set_xlabel('Soil Moisture %')
ax[1].set_ylabel('Frequency')
ax[1].set_title('Histogram of Soil Moisture (Testing)')
plt.tight_layout()
plt.show()


# Regression Using XGBoost

In [ ]:
# ---------------- TRAIN A REGRESSION MODEL USING XGBOOST ----------------------
import xgboost as xgb
import sklearn

# -------Function to compute regression metrics--------
def compute_regression_metrics(y_true, y_pred):
    mae = sklearn.metrics.mean_absolute_error(y_true, y_pred)
    r2 = sklearn.metrics.r2_score(y_true, y_pred)
    std_residuals = np.std(y_true - y_pred)

    diff = y_true - y_pred
    bias = np.mean(diff)
    rmse = np.sqrt(np.mean(diff ** 2))
    ubrmse = np.sqrt(np.mean((diff - bias) ** 2))
    return mae, r2, std_residuals, bias, rmse, ubrmse

params = {
    'objective': 'reg:squarederror',   # Loss function
    'eval_metric': 'mae',              # Mean aveage error
    'max_depth': 5,                    # Depth of trees
    'eta': 0.1,                        # Learning rate
    'seed': 42                         # For reproducibility
}

# Convert to DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=feature_names)
dtest = xgb.DMatrix(X_test, feature_names=feature_names)

# Train model
model = xgb.train(params, dtrain, num_boost_round=100)

# Predict on test data
y_pred = model.predict(dtest)

# Calculate metrics
y_true = y_test
residuals = y_true - y_pred
mae, r2, std_residuals, bias, rmse, ubrmse = compute_regression_metrics(y_true, y_pred)

#-------------------Plot results on TEST data------------------------
fig,ax = plt.subplots(1,2,figsize=(15,4))
# Regression Plot
ax[0].scatter(y_true, y_pred, alpha=0.5,edgecolors='black',label='Model Prediction') # model predictions
ax[0].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
ax[0].set_xlabel('Actual Soil Moisture (m³/m³)')
ax[0].set_ylabel('Predicted Soil Moisture (m³/m³)')
ax[0].set_title('Regression Plot')
textstr = f"MAE: {mae:.3f}\nR²: {r2:.3f}\nRMSE: {rmse:.3f}\nubRMSE: {ubrmse:.3f}"
ax[0].text(0.05, 0.95, textstr,
           transform=ax[0].transAxes,
           verticalalignment='top',
           horizontalalignment='left',
           fontsize=10)

# Residuals Plot
ax[1].scatter(y_pred, residuals, color='g',alpha=0.5,edgecolors='black')
ax[1].axhline(y=0, color='r', linestyle='--')
ax[1].set_xlabel("Predicted")
ax[1].set_ylabel("Residuals")
ax[1].set_title("Residual Plot")
textstr = f"Std Residuals: {std_residuals:.2f}"
ax[1].text(0.05, 0.95, textstr,
           transform=ax[1].transAxes,
           verticalalignment='top',
           horizontalalignment='left',
           fontsize=10)

plt.tight_layout()
plt.suptitle('Testing Data',fontsize=12,fontweight='bold')
plt.show()

#-------------------Plot results on REFERENCE data------------------------
# Calculate metrics
residuals = y_true - ref_test
mae, r2, std_residuals, bias, rmse, ubrmse = compute_regression_metrics(y_true, ref_test)


fig,ax = plt.subplots(1,2,figsize=(15,4))
# Regression Plot
ax[0].scatter(y_true, ref_test, alpha=0.5,edgecolors='black')
ax[0].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
ax[0].set_xlabel('Actual Soil Moisture (m³/m³)')
ax[0].set_ylabel('Predicted Soil Moisture (m³/m³)')
ax[0].set_title('Regression Plot')
textstr = f"MAE: {mae:.3f}\nR²: {r2:.3f}\nRMSE: {rmse:.3f}\nubRMSE: {ubrmse:.3f}"
ax[0].text(0.05, 0.95, textstr,
           transform=ax[0].transAxes,
           verticalalignment='top',
           horizontalalignment='left',
           fontsize=10)

# Residuals Plot
ax[1].scatter(ref_test, residuals, color='g',alpha=0.5,edgecolors='black')
ax[1].axhline(y=0, color='r', linestyle='--')
ax[1].set_xlabel("Predicted")
ax[1].set_ylabel("Residuals")
ax[1].set_title("Residual Plot")
textstr = f"Std Residuals: {std_residuals:.2f}"
ax[1].text(0.05, 0.95, textstr,
           transform=ax[1].transAxes,
           verticalalignment='top',
           horizontalalignment='left',
           fontsize=10)

plt.tight_layout()
plt.suptitle('Reference Data (SMAP)',fontsize=12,fontweight='bold')
plt.show()

#---------------------- Plot feature importance --------------------------------
ax = xgb.plot_importance(model, importance_type="gain",show_values=False)
fig = plt.gcf()
fig.set_size_inches(6,3)
plt.title("Feature Importance (Gain)")
plt.show()


print("MAE:", mean_absolute_error(y_true, y_pred))
print("R2 Score:", r2_score(y_true, y_pred))
print("Std Residuals:", np.std(residuals))


# Regression Using Multi-Layer Perceptron (MLP)

In [ ]:
# ------------------- REGRESSION USING MULTIPLE LAYER PERCEPTRON (MLP)-------------------
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.neural_network import MLPRegressor
import sklearn

# define a range of layer sizes each with one neuron
layerList = [
    (10,),  # 1 hidden layer, 10 neuron
    (10, 10),  # 2 hidden layers, 10 neuron each
    (10, 10, 10),  # 3 hidden layers, 10 neuron each
    (10, 10, 10, 10),  # 4 hidden layers, 10 neuron each
    (10, 10, 10, 10, 10)  # 5 hidden layers, 10 neuron each
]
r2_train_scores = []

fig1, ax1 = plt.subplots(1, len(layerList), figsize=(20, 5))
fig2, ax2 = plt.subplots(1, len(layerList), figsize=(20, 5))


for i,layer in enumerate(layerList):
    # initialize MLP regressor
    mlp = MLPRegressor(
        hidden_layer_sizes=layerList[i],
        activation='relu',
        solver='adam',
        max_iter=5000,
        random_state=42,
        learning_rate_init=0.001,
        alpha=0.01
        )

    # train the MLP
    mlp.fit(X_train, y_train)

    # predict on training set
    y_pred_train = mlp.predict(X_train)
    mae, r2, std_residuals, bias, rmse, ubrmse = compute_regression_metrics(y_train, y_pred_train)
    r2_train_scores.append(r2)

    # predict on test set
    y_pred_test = mlp.predict(X_test)
    mae, r2, std_residuals, bias, rmse, ubrmse = compute_regression_metrics(y_test, y_pred_test)
    print(f"Layer Size: {layer}")
    print("RMSE:", rmse)
    print("ubRMSE:", ubrmse)

    # regression plot per model
    ax1[i].scatter(y_test, y_pred_test, color="green", alpha=0.6, label="Predicted Soil Moisture")
    ax1[i].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "k--", lw=2,
                label='1:1 line')  # Identity line
    ax1[i].set_xlabel("Actual Soil Moisture")
    ax1[i].set_ylabel("Predicted Soil Moisture")
    ax1[i].set_title(f"Testing Set with {len(layer)} Layers")
    ax1[i].grid(True)
    # ax1[i].legend()

    # Display metrics on plot
    test_metrics = f"MAE: {mae:.4f}\nR²: {r2:.4f}\nStd Res: {std_residuals:.4f}"
    ax1[i].text(0.05, 0.95, test_metrics, transform=ax1[i].transAxes, fontsize=10,
                verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"))

    # residual plot per model
    ax2[i].scatter(y_pred_test, y_test - y_pred_test, color="green", alpha=0.6, label="Prediction Residuals")
    ax2[i].axhline(0, color="k", linestyle="--", lw=2, label='0 line')  # Horizontal line at y=0
    ax2[i].set_ylabel("Residuals")
    ax2[i].set_xlabel("Predicted Soil Moisture")
    ax2[i].set_title(f"Testing Set with {len(layer)} Layers")
    ax2[i].set_ylim(-0.4, 0.4)
    ax2[i].grid(True)
    # ax2[i].legend()

    # Display metrics on Test Set plot
    ax2[i].text(0.05, 0.95, test_metrics, transform=ax2[i].transAxes, fontsize=10,
                verticalalignment='top', bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"))

fig1.suptitle('Regression Plots for MLP Layers',fontweight='bold')
fig2.suptitle('Residual Plots for MLP Layers',fontweight='bold')

plt.tight_layout()
plt.show()

# plot the effect of accuracy vs number of hidden layers
plt.plot(list(range(1,len(layerList)+1)), r2_train_scores,linestyle="-", marker=".")
plt.axvline(list(range(1,len(layerList)+1))[np.argmax(r2_train_scores)], color="r", linestyle="--",label="Max Score")
plt.title("Training Accuracy for Number of MLP Layers")
plt.xlabel("Number of Layers")
plt.ylabel("R²")
plt.legend()
plt.show()

#--------------------- Plot the best performing MLP --------------------------
print(f"Best Model has {layerList[np.argmax(r2_train_scores)]} layers")

# initialize MLP regressor
mlp = MLPRegressor(
    hidden_layer_sizes=layerList[np.argmax(r2_train_scores)],
    activation='relu',
    solver='adam',
    max_iter=5000,
    random_state=42,
    learning_rate_init=0.001,
    alpha=0.01
    )

# train the MLP
mlp.fit(X_train, y_train)

# predict on test set
y_pred = mlp.predict(X_test)
mae, r2, std_residuals, bias, rmse, ubrmse = compute_regression_metrics(y_test, y_pred_test)

# -------------------- PLOT RESULTS ------------------
fig,ax = plt.subplots(1,2,figsize=(15,4))
# Regression Plot
ax[0].scatter(y_true, y_pred, alpha=0.5,edgecolors='black',label='Model Prediction') # model predictions
ax[0].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
ax[0].set_xlabel('Actual Soil Moisture (m³/m³)')
ax[0].set_ylabel('Predicted Soil Moisture (m³/m³)')
ax[0].set_title('Regression Plot')
textstr = f"MAE: {mae:.3f}\nR²: {r2:.3f}\nRMSE: {rmse:.3f}\nubRMSE: {ubrmse:.3f}"
ax[0].text(0.05, 0.95, textstr,
           transform=ax[0].transAxes,
           verticalalignment='top',
           horizontalalignment='left',
           fontsize=10)

# Residuals Plot
ax[1].scatter(y_pred, residuals, color='g',alpha=0.5,edgecolors='black')
ax[1].axhline(y=0, color='r', linestyle='--')
ax[1].set_xlabel("Predicted")
ax[1].set_ylabel("Residuals")
ax[1].set_title("Residual Plot")
textstr = f"Std Residuals: {std_residuals:.2f}"
ax[1].text(0.05, 0.95, textstr,
           transform=ax[1].transAxes,
           verticalalignment='top',
           horizontalalignment='left',
           fontsize=10)

plt.tight_layout()
plt.suptitle('Testing Data',fontsize=12,fontweight='bold')
plt.show()

# Regression Using a 1D CNN

In [ ]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn as nn
import sklearn
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# -------Function to compute regression metrics--------
def compute_regression_metrics(y_true, y_pred):
    mae = sklearn.metrics.mean_absolute_error(y_true, y_pred)
    r2 = sklearn.metrics.r2_score(y_true, y_pred)
    std_residuals = np.std(y_true - y_pred)

    diff = y_true - y_pred
    bias = np.mean(diff)
    rmse = np.sqrt(np.mean(diff ** 2))
    ubrmse = np.sqrt(np.mean((diff - bias) ** 2))
    return mae, r2, std_residuals, bias, rmse, ubrmse

# Read in extracted soil moisture data
data = pd.read_csv('/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/Extracted Data/soil_moisture_data.csv', parse_dates=['date'])

# Scale soil moisture values from percentage to volume ratio
data['soil_moisture'] = data['soil_moisture'] / 100
data['sm_smap'] = data['sm_smap'] / 100

# Separate into target and features
date = data['date']
target = data['soil_moisture'].to_numpy()
reference = data['sm_smap'].to_numpy()
features = data.drop(columns=['soil_moisture', 'date','sm_smap'])

feature_names = list(features.columns)
features = features.to_numpy()
print(len(feature_names))

print("Target Shape", target.shape)
print("Features Shape", features.shape)

# Training and testing splits
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Testing an validation splits
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Step 1: Standard scaling based on training data only
scaler = StandardScaler() # initialize the scaler
scaler.fit(X_train) # fit to only the training data
X_train = scaler.transform(X_train) # scale training data
X_val = scaler.transform(X_val) # scale validation data
X_test = scaler.transform(X_test) # scale test data

# Target (y) scaling
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_val = scaler_y.transform(y_val.reshape(-1, 1)).flatten()
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

#------------------- Create custom dataset class -----------------------
class SoilDataset(Dataset):
    def __init__(self, X, y):
      # Convert numpy arrays to torch tensors with the correct dtype
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X_sample = self.X[idx].unsqueeze(0)
        y_sample = self.y[idx].unsqueeze(0)
        return X_sample, y_sample

# Step 3: Instantiate datasets
train_dataset = SoilDataset(X_train, y_train)
val_dataset = SoilDataset(X_val, y_val)
test_dataset = SoilDataset(X_test, y_test)

# Step 4: Create DataLoaders
from torch.utils.data import DataLoader
batch_size = 64
num_workers = 2
prefetch_factor = 3
train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True,num_workers=num_workers,pin_memory=True,prefetch_factor=prefetch_factor)
val_loader = DataLoader(val_dataset, batch_size=batch_size,shuffle=True,num_workers=num_workers,pin_memory=True,prefetch_factor=prefetch_factor)
test_loader = DataLoader(test_dataset, batch_size=batch_size,shuffle=True,num_workers=num_workers,pin_memory=True,prefetch_factor=prefetch_factor)

print("train dataset size is:", len(train_dataset))
print("val dataset size is:", len(val_dataset))
print("test dataset size is:", len(test_dataset))

# use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

# Hyperparameter Tuning

In [ ]:
!pip install optuna

In [ ]:
import optuna
# ------------------- DEFINE OBJECTIVE FUNCTION FOR OPTUNA ---------------------
def objective(trial):
    # Fix random seed
    torch.manual_seed(15)
    np.random.seed(15)

    # Suggest hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-2,log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2,log=True)
    dropout_rate = trial.suggest_float('dropout', 0.0, 0.5)
    kernel_size = trial.suggest_int('kernel_size', 3, 7, step=2)
    channels_1 = trial.suggest_int('channels_1', 4, 32)
    channels_2 = trial.suggest_int('channels_2', 8, 64)
    channels_3 = trial.suggest_int('channels_3', 16, 128)
    gamma = trial.suggest_float('gamma', 0.1, 0.9)

    # Define model with variable architecture
    class TrialCNN(nn.Module):
        def __init__(self):
            super(TrialCNN, self).__init__()
            self.net = nn.Sequential(
                nn.Conv1d(1, channels_1, kernel_size=kernel_size, padding=kernel_size//2, stride=3), nn.ReLU(),
                nn.Conv1d(channels_1, channels_2, kernel_size=kernel_size, padding=kernel_size//2, stride=3), nn.ReLU(),
                nn.Conv1d(channels_2, channels_3, kernel_size=kernel_size, padding=kernel_size//2, stride=3), nn.ReLU(),
                nn.AdaptiveAvgPool1d(1),
                nn.Flatten(),
                nn.Dropout(dropout_rate),
                nn.Linear(channels_3, 1)
            )

        def forward(self, x):
            return self.net(x)

    model = TrialCNN().to(device) # send to gpu
    criterion = nn.MSELoss()

    # Adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=gamma)

   # ----------------------- Training loop ----------------------------
    best_val_loss = float('inf')
    for epoch in range(10):
        model.train() # set model to training mode
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad() # reset the gradient
            outputs = model(X_batch) # make predictions
            loss = criterion(outputs, y_batch) # calculate loss
            loss.backward() # backpropogation
            optimizer.step() # make a step
        scheduler.step() # reduce the learning rate

      # -------------------------- Validation -----------------------------
        model.eval() # set model to evaluation mode
        val_losses = [] # initialize validation loss
        with torch.no_grad(): # do not track gradients
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                val_loss = criterion(outputs, y_batch).item()
                val_losses.append(val_loss) # store the loss for each minibatch

        val_loss_mean = np.mean(val_losses) # average the validation loss


        if val_loss_mean < best_val_loss:
            best_val_loss = val_loss_mean

    return best_val_loss


In [ ]:
# ------------------------ OPTUNA STUDY ----------------------------------------

# Create Optuna study to minimize the loss
study = optuna.create_study(direction='minimize')

# Optimize using the objective funtion
study.optimize(objective, n_trials=50)

print("Best Trial:")
print(study.best_trial)

# Store best parameters
best_params = study.best_params
lr = best_params['lr']
weight_decay = best_params['weight_decay']
dropout_rate = best_params['dropout']
kernel_size = best_params['kernel_size']
channels_1 = best_params['channels_1']
channels_2 = best_params['channels_2']
channels_3 = best_params['channels_3']
gamma = best_params['gamma']

# Print best values
print("Best Hyperparameters:")
print(f"Learning Rate: {lr}")
print(f"Weight Decay: {weight_decay}")
print(f"Dropout Rate: {dropout_rate}")
print(f"Kernel Size: {kernel_size}")
print(f"Channels 1: {channels_1}")
print(f"Channels 2: {channels_2}")
print(f"Channels 3: {channels_3}")
print(f"Gamma: {gamma}")

In [ ]:
# ----------------- TRAIN MODEL WITH TUNED HYPERPARAMETERS --------------------

class BestSpectralCNN(nn.Module):
    def __init__(self):
        super(BestSpectralCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv1d(1, channels_1, kernel_size=kernel_size, stride=3, padding=kernel_size // 2), nn.ReLU(),
            nn.Conv1d(channels_1, channels_2, kernel_size=kernel_size, stride=3, padding=kernel_size // 2), nn.ReLU(),
            nn.Conv1d(channels_2, channels_3, kernel_size=kernel_size, stride=3, padding=kernel_size // 2), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Dropout(dropout_rate),
            nn.Linear(channels_3, 1)
        )

    def forward(self, x):
        return self.net(x)

model = BestSpectralCNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=gamma)
criterion = nn.MSELoss()

# ----------------------- Training loop ----------------------------
# keeping track of loss
val_loss_history = []
train_loss_history = []

# For early stopping
no_improvement = 0

best_val_loss = float('inf')
for epoch in range(50):
    model.train() # set model to training mode
    train_losses = []
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad() # reset the gradient
        outputs = model(X_batch) # make predictions
        loss = criterion(outputs, y_batch) # calculate loss
        loss.backward() # backpropogation
        optimizer.step() # make a step
        train_losses.append(loss.item()) # keep track of training loss

    train_loss_mean = np.mean(train_losses)
    train_loss_history.append(train_loss_mean)

    scheduler.step() # reduce the learning rate

# -------------------------- Validation -----------------------------
    model.eval() # set model to evaluation mode
    val_losses = [] # initialize validation loss
    with torch.no_grad(): # do not track gradients
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            val_loss = criterion(outputs, y_batch).item()
            val_losses.append(val_loss) # store the loss for each minibatch

    val_loss_mean = np.mean(val_losses)
    val_loss_history.append(val_loss_mean) # store val loss for epoch

    if val_loss_mean < best_val_loss:
        best_val_loss = val_loss_mean
        best_epoch = epoch
        torch.save(model.state_dict(), "best_model.pt") # save best model
        no_improvement = 0
    else:
      no_improvement = no_improvement + 1
      if no_improvement >= 15: # early stopping
        print(f"\nEarly stopping triggered after {epoch+1} epochs")
        break

    print(f"Epoch {epoch+1}, Val Loss: {val_loss_mean:.4f}")


# ---------- PLOT LOSS -------------
import matplotlib.pyplot as plt

plt.plot(train_loss_history, '.-',label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot validation loss
plt.plot(val_loss_history, 'g.-',label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# ---------------- TEST SET RESULTS ON OPTIMAL MODEL ---------------------------
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# loading the best model
model = BestSpectralCNN().to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/Imaging Science MS/Applied ML for Remote Sensing/Project/Data/Models/best_model.pt'))
model.eval()

all_preds = []
all_targets = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        preds = model(X_batch)
        all_preds.append(preds.cpu().numpy())
        all_targets.append(y_batch.cpu().numpy())

all_preds = np.concatenate(all_preds).flatten()
all_targets = np.concatenate(all_targets).flatten()

# Unscale predictions and targets before evaluation
all_preds = scaler_y.inverse_transform(all_preds.reshape(-1, 1)).flatten()
all_targets = scaler_y.inverse_transform(all_targets.reshape(-1, 1)).flatten()

# Calculate metrics
y_true = all_targets
y_pred = all_preds
residuals = y_true - y_pred
mae, r2, std_residuals, bias, rmse, ubrmse = compute_regression_metrics(y_true, y_pred)

print("MAE:", mean_absolute_error(y_true, y_pred))
print("R2 Score:", r2_score(y_true, y_pred))
print("Std Residuals:", std_residuals)
print("RMSE:", rmse)
print("ubRMSE:", ubrmse)
print("Bias:", bias)

# ---------------------- PLOT TEST RESULTS ON OPTIMAL MODEL --------------------
fig,ax = plt.subplots(1,2,figsize=(15,4))
# Regression Plot
ax[0].scatter(y_true, y_pred, alpha=0.5,edgecolors='black')
ax[0].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
ax[0].set_xlabel('Actual Soil Moisture (m³/m³)')
ax[0].set_ylabel('Predicted Soil Moisture (m³/m³)')
ax[0].set_title('Regression Plot')
textstr = f"MAE: {mae:.3f}\nR²: {r2:.3f}\nRMSE: {rmse:.3f}\nubRMSE: {ubrmse:.3f}"
ax[0].text(0.05, 0.95, textstr,
           transform=ax[0].transAxes,
           verticalalignment='top',
           horizontalalignment='left',
           fontsize=10)

# Residuals Plot
ax[1].scatter(y_pred, residuals, color='g',alpha=0.5,edgecolors='black')
ax[1].axhline(y=0, color='r', linestyle='--')
ax[1].set_xlabel("Predicted")
ax[1].set_ylabel("Residuals")
ax[1].set_title("Residual Plot")
textstr = f"Std Residuals: {std_residuals:.2f}"
ax[1].text(0.05, 0.95, textstr,
           transform=ax[1].transAxes,
           verticalalignment='top',
           horizontalalignment='left',
           fontsize=10)

plt.tight_layout()
plt.suptitle('Testing Data',fontsize=12,fontweight='bold')
plt.show()